In [ ]:
import sys

sys.path.append('c:\\users\\pabas\\appdata\\local\\programs\\python\\python311\\lib\\site-packages')


In [1]:
import numpy as np
import cv2

# Initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open webcam video stream
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Create a window to display the video feed
cv2.namedWindow("Video Feed")

# Initialize head count and a list to keep track of detected heads
head_count = 0
detected_heads = []

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is captured successfully
    if not ret:
        print("Error: Could not read frame.")
        break

    # Resizing for faster detection
    frame = cv2.resize(frame, (640, 480))

    # Detect people in the image
    # Returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(frame, winStride=(8, 8))

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

    for (xA, yA, xB, yB) in boxes:
        # Check if the head has been detected before
        new_head = True
        for (x, y, _, _) in detected_heads:
            if abs(xA - x) < 20 and abs(yA - y) < 20:
                new_head = False
                break

        if new_head:
            # Draw a green rectangle around the head
            cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
            # Increment head count
            head_count += 1
            # Add the detected head to the list
            detected_heads.append((xA, yA, xB, yB))

    # Display the head count in the top-left corner
    cv2.putText(frame, f'Heads: {head_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow("Video Feed", frame)

    # Write the output video (optional)
    # out.write(frame.astype('uint8'))

    # Press 'q' to exit the webcam feed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the window
cap.release()
cv2.destroyAllWindows()
